In [ ]:
!apt-get install python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr \
flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig

In [ ]:
!pip install docx2txt
!pip install validate-docbr
!pip install pycep-correios
!pip install phonenumbers
!pip install textract

In [ ]:
from validate_docbr import CPF, CNH, CNPJ, PIS, TituloEleitoral

import docx2txt
import re
import pycep_correios
import phonenumbers
import textract
import os

In [ ]:
def validate_cpf(doc):
    cpf = CPF()
    return cpf.validate(doc)

In [ ]:
def validate_cnh(doc):
    cnh = CNH()
    return cnh.validate(doc)

In [ ]:
def validate_cnpj(doc):
    cnpj = CNPJ()
    return cnpj.validate(doc)

In [ ]:
def validate_pis(doc):
    pis = PIS()
    return pis.validate(doc)

In [ ]:
def validate_titulo(doc):
    titulo = TituloEleitoral()
    return titulo.validate(doc)

In [ ]:
def validate_cep(cep):
  if re.match("[0-9]{5}-[0-9]{3}", cep) or re.match("[0-9]{5}[0-9]{3}", cep):
    endereco = pycep_correios.get_address_from_cep(cep)
    return endereco
  return False

In [ ]:
def validate_celular(telefone):
  if not telefone[:3] == "+55":
    telefone = "+55" + telefone

  z = phonenumbers.parse(telefone, None)

  return phonenumbers.is_valid_number(z)


In [ ]:
def verificacoes(texto):
  lines = texto.split('\n')

  for line in lines:
    if line:
      for word in line.split(' '):
        if word == '':
          continue
        try:
          word = word.strip()

          if validate_cpf(word):
            print("tem cpf")

          if len(word) == 9:
            endereco = validate_cep(word)
            if endereco:
              print("tem cep", endereco)

          if validate_celular(word):
            print("tem celular")

          if validate_cnh(word):
            print("tem cnh")

          if validate_cnpj(word):
            print("tem cnpj")

          if validate_pis(word):
            print("tem pis")

          if validate_titulo(word):
            print("tem titulo")
        except:
          continue

In [ ]:
def verificar_docx(arquivo):
  result = docx2txt.process(arquivo)
  verificacoes(result)



In [ ]:
def verificar_txt(arquivo):
  result = open(arquivo, "r")
  verificacoes(result.read())

In [ ]:
def verificar_pdf(arquivo):
  texto = textract.process(arquivo)
  texto = texto.decode("utf-8")
  verificacoes(texto)

In [ ]:
def verificar_documento(arquivo):
  arquivo_partes = arquivo.split('.')
  ext = arquivo_partes[len(arquivo_partes)-1]

  if os.path.isfile(arquivo):
    print("Verificando -- " + arquivo)
    if ext.lower() == 'txt':
      verificar_txt(arquivo)
    elif ext.lower() == 'pdf':
      verificar_pdf(arquivo)
    elif ext.lower() == 'docx':
      verificar_docx(arquivo)
    else:
      print('Tipo inválido')

    print("--------------------------FIM-----------------------------")
  else:
    print('Arquivo não encontrado.')

In [ ]:
def verificar_diretorio():
  files = os.listdir()
  for arquivo in files:
    verificar_documento(arquivo)

In [ ]:
verificar_diretorio()